<h1 style="text-align:center; color:black; text-decoration:underline;">Préparation des données pour le projet de cartographie des données validations sur les lignes</h1>

<h2 style="text-decoration:bold; color:black;">Source de données</h2>
<ul>
    <a href="https://data.iledefrance-mobilites.fr/explore/dataset/arrets/export/"><li>Référentiel des arrêts de bud IDFM</li></a>
    <a href="https://data.iledefrance.fr/explore/dataset/geoflar-departements/information/"><li>Contour des départements, GEOFLA</li></a>
    <a href="https://www.insee.fr/fr/statistiques/7739582?sommaire=7728826#consulter"><li>Population légale 2021</li></a>
</ul>

<h2 style="text-decoration:bold; color:black;">Notebook</h2>

In [ ]:
import pandas 
import os 
os.environ["USE_PYGEOS"] = "0"
import geopandas

In [ ]:
##### PREPARATION DES DONNEES DES COMMUNES #####
com = geopandas.read_file(r"..\DONNEES\communes_idf_2022.shp", encoding="utf-8")
print(com.crs) #Vérifier la projection.
com.to_crs(epsg="3949", inplace=True) #Convertir en mètre.
print(com.crs) #Vérifier la projection.

com.info()

valdoise = com[ com["numdep"] == "95" ]
valdoise.drop(labels=["shape_leng", "fusioinsee", "st_areasha", "st_lengths", "nomcomto"], axis=1, inplace=True)
valdoise.reset_index(drop=True, inplace=True)
valdoise.head()

save = False 

if save == True :
    valdoise.to_file(filename=r"..\TRAITEMENT\communes_valdoise_2022.geojson", driver="GeoJSON")


In [ ]:
##### PREPARATION DES DONNEES D'ARRETS #####

arrets = geopandas.read_file(filename=r"..\DONNEES\arrets.geojson", geometry="geometry", encoding="utf-8")
#Changer la projection si nécessaire 
def convert_crs(gdf, crs="3949") :
    if gdf.crs != crs :
        print(gdf.crs)
        gdf.to_crs(epsg=crs, inplace=True)
        print(gdf.crs)
    else :
        print(gdf.crs)

    return gdf

arrets = convert_crs(arrets)

#Récupérer les arrêts dans le Val d'Oise 
arrets_vdo = geopandas.sjoin(left_df=arrets, right_df=valdoise, predicate="within")
#arrets_vdo.explore() #Vérifier le résultat.

#Conserver uniquement les colonnes qui nous intéressent :
print(arrets_vdo.columns)
arrets_vdo.drop(labels=['arrcreated', 'arrchanged', 'arrtown',
                        'arrpostalregion', 'arraccessibility', 'arraudiblesignals',
                        'arrvisualsigns', 'arrfarezone', 'index_right'], 
                        axis=1, inplace=True)

arrets_vdo = arrets_vdo[ arrets_vdo["arrtype"] == "bus" ]
arrets_vdo.drop_duplicates(subset="zdaid", inplace=True)

arrets_vdo.to_file(filename=r"..\TRAITEMENT\arrets_bus_vdo.geojson", driver="GeoJSON", encoding="utf-8")

In [ ]:
##### PREPARER LES DONNEES DEMOGRAPHIQUES #####

pop21 = pandas.read_csv(r"..\DONNEES\donnees_communes.csv", sep=";", decimal=",", encoding="utf-8")

pop_vdo = pop21[ pop21["DEP"] == "95" ]
pop_vdo.reset_index(drop=True, inplace=True)

def To_String(r) :
    res = r["COM"]
    res = str(res)

    return res

pop_vdo["COM"] = pop_vdo.apply(func=To_String, axis=1)

pop_vdo.to_csv(r"..\TRAITEMENT\population_2021.csv", 
               sep=",", decimal=".", encoding="utf-8")

pop_vdo.info()